In [1]:
from deap import base, creator, tools
import random

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [2]:
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

In [10]:
import numpy as np
pop = toolbox.population(n=50)
CXPB, MUTPB, NGEN = 0.5, 0.2, 40
print(np.matrix(pop))

[[1.47952635e-01 9.05617313e-01 5.77805471e-01 3.37513963e-01
  7.74149715e-01 1.84927917e-01 3.57708393e-01 7.63308268e-01
  8.54817001e-01 7.44880608e-02]
 [3.78266083e-01 8.25593621e-01 2.57441343e-02 7.18210616e-01
  8.95846704e-01 1.82787932e-02 9.97170645e-01 6.80981478e-01
  9.62486639e-01 2.40932380e-02]
 [1.21612485e-01 4.21357112e-01 1.32384340e-01 5.61135216e-02
  3.50643476e-01 8.50519872e-03 2.76696948e-03 3.84962520e-02
  2.61099015e-01 3.41679163e-01]
 [9.24897353e-02 8.21066823e-01 7.36555569e-01 2.57694587e-01
  4.80480107e-01 3.89890790e-01 8.23490338e-01 6.03383348e-01
  7.14756848e-01 1.68446473e-01]
 [3.17858951e-01 6.19410019e-01 9.82169076e-01 5.01275051e-01
  7.92821436e-01 5.83445264e-01 3.51378372e-01 3.07108035e-01
  6.75766216e-01 2.67494182e-01]
 [5.58345782e-01 9.83112139e-01 1.01349758e-01 2.47897416e-01
  4.63128146e-01 9.12827749e-01 8.17642286e-02 6.21112897e-01
  8.55164581e-01 1.13840929e-01]
 [9.43237805e-01 1.95701572e-01 3.41047071e-01 2.83146936e

In [11]:
# Evaluate the entire population
fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

for g in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring, https://stackoverflow.com/questions/6800481/python-map-object-is-not-subscriptable
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

# The population is entirely replaced by the offspring
pop[:] = offspring
print(np.matrix(pop))

[[ 0.07418791  0.41823216  0.18773095  0.12659923  0.66809678  0.77495921
   0.67057691  0.22096036  0.76386866  0.29814941]
 [ 0.12161249  0.42135711  0.13238434  0.05611352  0.35064348  0.0085052
   0.00276697  0.03849625  0.26109901  0.34167916]
 [ 0.52332069  0.66990403  0.38865441  0.58973981  0.22710728  0.20448631
   0.68441708  0.3241301   0.14938411  0.182733  ]
 [ 0.18748298  0.28225817  0.79966661  0.14905853  0.2143925   0.87639785
   0.77069934  0.33334602  0.06615449  0.03891499]
 [ 0.7366071   0.24435448  0.91277053  0.44151018  0.65309474  0.97718102
   0.70966094  0.01293359  0.48483159  0.75666552]
 [ 0.244343    0.48258466  0.04182729  0.68010618  0.50456061  0.9771864
   0.21263122  0.33556384  0.61276067  0.3685419 ]
 [ 0.36397897  0.94661065  0.00794253  0.04677552  0.00462733  0.22812341
   0.62727786  0.33807873  0.44514548  0.5426966 ]
 [ 0.97113465  0.47925999  0.00684196  0.3301498   0.12042434  0.22509161
   0.08861157  0.61928824  0.59185706  0.92356222]
 [